In [ ]:
from google.colab import files
uploaded = files.upload()  # Manually upload the file


Saving ML_DEG_dataset.csv to ML_DEG_dataset.csv


In [ ]:
from google.colab import files
uploaded = files.upload()  # Manually upload the file

Saving GSE21815_gene_expression_renamed.csv to GSE21815_gene_expression_renamed.csv


In [ ]:
pip install xgboost

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('ML_DEG_dataset.csv')

In [ ]:
df.head()

,geo_accession,target,TUBB,CDT1,LPP,UGP2,MFN2,HIST1H2BD,SEMA3E,RGS5,...,LCAT,MCM5,ASPM,HMGCS2,AGT,CCDC68,CD46,PLCG2,SOX4,NUP107
0,GSM277495,0,1971.4430,35.27089,8473.877,61.36008,878.7402,3212.8910,-16.952740,403.8409,...,79.59111,224.2105,144.5214,18967.55000,143.8013,624.58200,2115.487,1365.6700,426.4089,419.1906
1,GSM277496,1,606.7773,468.34940,5391.271,30.43565,715.1648,323.2217,-8.576768,615.3448,...,94.12312,361.0414,1086.0480,96.36639,235.3114,58.14994,5566.119,303.0632,1563.1800,1017.3650
2,GSM277497,0,8916.9360,82.57671,27863.030,34.52829,1160.9170,778.4032,52.338870,1263.4070,...,217.14050,288.5819,352.1789,11035.48000,115.9880,126.39290,1498.417,810.1133,590.7308,619.0782
3,GSM277498,1,3148.3370,580.13820,5325.731,36.86857,470.3635,328.6313,-10.858900,124.0610,...,47.98767,555.0460,1424.9820,14479.59000,152.0744,172.29320,3903.002,181.3151,1438.3140,1847.4700
4,GSM277499,0,3906.8750,93.08880,20692.270,64.22387,1166.8960,1181.2000,-0.608158,734.8578,...,214.11700,189.5398,141.4275,31952.92000,154.1114,201.99280,2057.749,543.7367,535.5609,462.7539


In [ ]:
pip install xgboost


In [ ]:
import pandas as pd
import numpy as np
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
import warnings
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, StackingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, StandardScaler
import warnings

warnings.filterwarnings("ignore")
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import matthews_corrcoef




warnings.filterwarnings("ignore")
path = "ML_DEG_dataset.csv"
df_train = pd.read_csv("ML_DEG_dataset.csv")
#columns_of_interest = ['MT1M', 'ITM2A', 'VSNL1', 'BRCA2', 'AFF3', 'LGALS2', 'PCSK2', 'TMEM100', 'CDH3', 'LIFR', 'FAM107A', 'STMN2', 'ABCA8', 'CENPA', 'ADH1B','target']

#df_train = df_train[columns_of_interest]
df_train = df_train.iloc[:, 1:]
df_train = df_train[[col for col in df_train.columns if col != 'target'] + ['target']]
X_train = df_train.iloc[:, 0:-1]
y_train = df_train.iloc[:, -1]

path = "GSE44076_gene_expression_renamed.csv"
df_test = pd.read_csv("GSE21815_gene_expression_renamed.csv")
df_test = df_test.iloc[:, 1:]
df_test = df_test[[col for col in df_test.columns if col != 'target'] + ['target']]

X_test = df_test.iloc[:, 0:-1]
y_test = df_test.iloc[:, -1]

common_cols = set(X_train.columns).intersection(X_test.columns)
X_train_common = X_train[list(common_cols)]
X_test_common = X_test[list(common_cols)]


# Validation dataset
validation_size = 0.30
seed = 7

# Define base models
models = {
    'LR': LogisticRegression(max_iter=1000),
    'KNN': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Bagging': BaggingClassifier(),
    'Extra Trees': ExtraTreesClassifier(),
    'XGBoost':XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

# Evaluate models
for name, model in models.items():
    kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
    cv_results = cross_val_score(model, X_train_common, y_train, cv=kfold, scoring='accuracy')
    print(f"{name} Training Accuracy: {cv_results.mean() * 100:.2f}%")

    # Train and test each model
    model.fit(X_train_common, y_train)
    predictions = model.predict(X_test_common)
    # Calculate metrics
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted', zero_division=0)
    recall = recall_score(y_test, predictions, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predictions, average='weighted', zero_division=0)
    mcc = matthews_corrcoef(y_test, predictions)

    # Print results
    print(f"{name} Testing Accuracy: {accuracy * 100:.2f}%")
    print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1-Score: {f1:.2f}, MCC: {mcc:.2f}")
    test_accuracy = accuracy_score(y_test, predictions)
    print(f"{name} Testing Accuracy: {test_accuracy * 100:.2f}%")

    # Print confusion matrix and classification report
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, predictions))
    print("Classification Report:")
    print(classification_report(y_test, predictions))

LR Training Accuracy: 93.63%
LR Testing Accuracy: 26.24%
Precision: 0.94, Recall: 0.26, F1-Score: 0.34, MCC: 0.13
LR Testing Accuracy: 26.24%
Confusion Matrix:
[[  9   0]
 [104  28]]
Classification Report:
              precision    recall  f1-score   support

           0       0.08      1.00      0.15         9
           1       1.00      0.21      0.35       132

    accuracy                           0.26       141
   macro avg       0.54      0.61      0.25       141
weighted avg       0.94      0.26      0.34       141

KNN Training Accuracy: 93.61%
KNN Testing Accuracy: 98.58%
Precision: 0.99, Recall: 0.99, F1-Score: 0.99, MCC: 0.88
KNN Testing Accuracy: 98.58%
Confusion Matrix:
[[  8   1]
 [  1 131]]
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.89      0.89         9
           1       0.99      0.99      0.99       132

    accuracy                           0.99       141
   macro avg       0.94      0.94      0

# Hub Genes

In [ ]:
import pandas as pd
import numpy as np
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
import warnings
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, StackingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import matthews_corrcoef
from xgboost import XGBClassifier




from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, StandardScaler
import warnings

warnings.filterwarnings("ignore")


warnings.filterwarnings("ignore")
path = "ML_DEG_dataset.csv"
df_train = pd.read_csv("ML_DEG_dataset.csv")
columns_of_interest = ['EHD2', 'TUBB', 'CSRP1', 'DPYSL3', 'TIMP2', 'RHOQ', 'ILK', 'SPARCL1', 'PALLD', 'PTRF',
                       'PPP3CB', 'MEF2C', 'RASL12', 'FLNA', 'PRNP', 'CAV2', 'SNCA', 'TNS1', 'TGFB1I1',
                       'SLIT2', 'ITPR1', 'LAMB2', 'RHOB', 'ACTN1','target']

df_train = df_train[columns_of_interest]
#df_train = df_train.iloc[:, 1:]
df_train = df_train[[col for col in df_train.columns if col != 'target'] + ['target']]
X_train = df_train.iloc[:, 0:-1]
y_train = df_train.iloc[:, -1]

path = "GSE44076_gene_expression_renamed.csv"
df_test = pd.read_csv("GSE21815_gene_expression_renamed.csv")
df_test = df_test.iloc[:, 1:]
df_test = df_test[[col for col in df_test.columns if col != 'target'] + ['target']]

X_test = df_test.iloc[:, 0:-1]
y_test = df_test.iloc[:, -1]

common_cols = set(X_train.columns).intersection(X_test.columns)
X_train_common = X_train[list(common_cols)]
X_test_common = X_test[list(common_cols)]


# Validation dataset
validation_size = 0.30
seed = 7

# Define base models
models = {
    'LR': LogisticRegression(max_iter=1000),
    'KNN': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Bagging': BaggingClassifier(),
    'Extra Trees': ExtraTreesClassifier(),
    'XGBoost':XGBClassifier(use_label_encoder=False, eval_metric='logloss')

}

# Evaluate models
for name, model in models.items():
    kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
    cv_results = cross_val_score(model, X_train_common, y_train, cv=kfold, scoring='accuracy')
    print(f"{name} Training Accuracy: {cv_results.mean() * 100:.2f}%")

    # Train and test each model
    model.fit(X_train_common, y_train)
    predictions = model.predict(X_test_common)
    # Calculate metrics
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted', zero_division=0)
    recall = recall_score(y_test, predictions, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predictions, average='weighted', zero_division=0)
    mcc = matthews_corrcoef(y_test, predictions)

    # Print results
    print(f"{name} Testing Accuracy: {accuracy * 100:.2f}%")
    print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1-Score: {f1:.2f}, MCC: {mcc:.2f}")
    test_accuracy = accuracy_score(y_test, predictions)
    print(f"{name} Testing Accuracy: {test_accuracy * 100:.2f}%")

    # Print confusion matrix and classification report
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, predictions))
    print("Classification Report:")
    print(classification_report(y_test, predictions))
    print("-" * 60)

LR Training Accuracy: 90.74%
LR Testing Accuracy: 95.04%
Precision: 0.94, Recall: 0.95, F1-Score: 0.94, MCC: 0.48
LR Testing Accuracy: 95.04%
Confusion Matrix:
[[  3   6]
 [  1 131]]
Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.33      0.46         9
           1       0.96      0.99      0.97       132

    accuracy                           0.95       141
   macro avg       0.85      0.66      0.72       141
weighted avg       0.94      0.95      0.94       141

------------------------------------------------------------
KNN Training Accuracy: 83.14%
KNN Testing Accuracy: 85.11%
Precision: 0.90, Recall: 0.85, F1-Score: 0.87, MCC: 0.16
KNN Testing Accuracy: 85.11%
Confusion Matrix:
[[  3   6]
 [ 15 117]]
Classification Report:
              precision    recall  f1-score   support

           0       0.17      0.33      0.22         9
           1       0.95      0.89      0.92       132

    accuracy                     

# For second Dataset

# 2. Pearson Correlation Feature Selection


# For 1st Dataset

In [ ]:
import pandas as pd
import numpy as np
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
import warnings
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, StackingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import matthews_corrcoef
from xgboost import XGBClassifier




from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, StandardScaler
import warnings

warnings.filterwarnings("ignore")


warnings.filterwarnings("ignore")
path = "ML_DEG_dataset.csv"
df_train = pd.read_csv("ML_DEG_dataset.csv")
columns_of_interest = ['MT1M', 'ITM2A', 'VSNL1', 'BRCA2', 'AFF3', 'LGALS2', 'PCSK2', 'TMEM100', 'CDH3', 'LIFR', 'FAM107A', 'STMN2', 'ABCA8', 'CENPA', 'ADH1B','target']

df_train = df_train[columns_of_interest]
#df_train = df_train.iloc[:, 1:]
df_train = df_train[[col for col in df_train.columns if col != 'target'] + ['target']]
X_train = df_train.iloc[:, 0:-1]
y_train = df_train.iloc[:, -1]

path = "GSE44076_gene_expression_renamed.csv"
df_test = pd.read_csv("GSE21815_gene_expression_renamed.csv")
df_test = df_test.iloc[:, 1:]
df_test = df_test[[col for col in df_test.columns if col != 'target'] + ['target']]

X_test = df_test.iloc[:, 0:-1]
y_test = df_test.iloc[:, -1]

common_cols = set(X_train.columns).intersection(X_test.columns)
X_train_common = X_train[list(common_cols)]
X_test_common = X_test[list(common_cols)]


# Validation dataset
validation_size = 0.30
seed = 7

# Define base models
models = {
    'LR': LogisticRegression(max_iter=1000),
    'KNN': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Bagging': BaggingClassifier(),
    'Extra Trees': ExtraTreesClassifier(),
    'XGBoost':XGBClassifier(use_label_encoder=False, eval_metric='logloss')

}

# Evaluate models
for name, model in models.items():
    kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
    cv_results = cross_val_score(model, X_train_common, y_train, cv=kfold, scoring='accuracy')
    print(f"{name} Training Accuracy: {cv_results.mean() * 100:.2f}%")

    # Train and test each model
    model.fit(X_train_common, y_train)
    predictions = model.predict(X_test_common)
    # Calculate metrics
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted', zero_division=0)
    recall = recall_score(y_test, predictions, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predictions, average='weighted', zero_division=0)
    mcc = matthews_corrcoef(y_test, predictions)

    # Print results
    print(f"{name} Testing Accuracy: {accuracy * 100:.2f}%")
    print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1-Score: {f1:.2f}, MCC: {mcc:.2f}")
    test_accuracy = accuracy_score(y_test, predictions)
    print(f"{name} Testing Accuracy: {test_accuracy * 100:.2f}%")

    # Print confusion matrix and classification report
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, predictions))
    print("Classification Report:")
    print(classification_report(y_test, predictions))
    print("-" * 60)


LR Training Accuracy: 92.44%
LR Testing Accuracy: 82.27%
Precision: 0.95, Recall: 0.82, F1-Score: 0.86, MCC: 0.46
LR Testing Accuracy: 82.27%
Confusion Matrix:
[[  9   0]
 [ 25 107]]
Classification Report:
              precision    recall  f1-score   support

           0       0.26      1.00      0.42         9
           1       1.00      0.81      0.90       132

    accuracy                           0.82       141
   macro avg       0.63      0.91      0.66       141
weighted avg       0.95      0.82      0.86       141

------------------------------------------------------------
KNN Training Accuracy: 92.74%
KNN Testing Accuracy: 53.90%
Precision: 0.94, Recall: 0.54, F1-Score: 0.64, MCC: 0.25
KNN Testing Accuracy: 53.90%
Confusion Matrix:
[[ 9  0]
 [65 67]]
Classification Report:
              precision    recall  f1-score   support

           0       0.12      1.00      0.22         9
           1       1.00      0.51      0.67       132

    accuracy                         

# 3. Lasso Regression Feature Selection

# For 1st Dataset

In [ ]:
import pandas as pd
import numpy as np
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import warnings
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, StackingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, StandardScaler
import warnings

warnings.filterwarnings("ignore")


warnings.filterwarnings("ignore")
path = "ML_DEG_dataset.csv"
df_train = pd.read_csv("ML_DEG_dataset.csv")
columns_of_interest = ['NR3C2', 'BRCA2', 'AFF3', 'LGALS2', 'TMEM100', 'CDH3', 'ABCG2', 'STMN2', 'ABCA8', 'AZGP1', 'TSPAN7', 'ADH1B','target']

df_train = df_train[columns_of_interest]
#df_train = df_train.iloc[:, 1:]
df_train = df_train[[col for col in df_train.columns if col != 'target'] + ['target']]
X_train = df_train.iloc[:, 0:-1]
y_train = df_train.iloc[:, -1]

path = "GSE44076_gene_expression_renamed.csv"
df_test = pd.read_csv("GSE21815_gene_expression_renamed.csv")
df_test = df_test.iloc[:, 1:]
df_test = df_test[[col for col in df_test.columns if col != 'target'] + ['target']]

X_test = df_test.iloc[:, 0:-1]
y_test = df_test.iloc[:, -1]

common_cols = set(X_train.columns).intersection(X_test.columns)
X_train_common = X_train[list(common_cols)]
X_test_common = X_test[list(common_cols)]


# Validation dataset
validation_size = 0.30
seed = 7

# Define base models
models = {
    'LR': LogisticRegression(max_iter=1000),
    'KNN': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Bagging': BaggingClassifier(),
    'Extra Trees': ExtraTreesClassifier(),
    'XGBoost':XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

# Evaluate models
for name, model in models.items():
    kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
    cv_results = cross_val_score(model, X_train_common, y_train, cv=kfold, scoring='accuracy')
    print(f"{name} Training Accuracy: {cv_results.mean() * 100:.2f}%")

    # Train and test each model
    model.fit(X_train_common, y_train)
    predictions = model.predict(X_test_common)
    # Calculate metrics
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted', zero_division=0)
    recall = recall_score(y_test, predictions, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predictions, average='weighted', zero_division=0)
    mcc = matthews_corrcoef(y_test, predictions)

    # Print results
    print(f"{name} Testing Accuracy: {accuracy * 100:.2f}%")
    print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1-Score: {f1:.2f}, MCC: {mcc:.2f}")
    test_accuracy = accuracy_score(y_test, predictions)
    print(f"{name} Testing Accuracy: {test_accuracy * 100:.2f}%")

    # Print confusion matrix and classification report
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, predictions))
    print("Classification Report:")
    print(classification_report(y_test, predictions))
    print("-" * 60)

LR Training Accuracy: 93.61%
LR Testing Accuracy: 96.45%
Precision: 0.98, Recall: 0.96, F1-Score: 0.97, MCC: 0.79
LR Testing Accuracy: 96.45%
Confusion Matrix:
[[  9   0]
 [  5 127]]
Classification Report:
              precision    recall  f1-score   support

           0       0.64      1.00      0.78         9
           1       1.00      0.96      0.98       132

    accuracy                           0.96       141
   macro avg       0.82      0.98      0.88       141
weighted avg       0.98      0.96      0.97       141

------------------------------------------------------------
KNN Training Accuracy: 93.89%
KNN Testing Accuracy: 73.05%
Precision: 0.95, Recall: 0.73, F1-Score: 0.80, MCC: 0.37
KNN Testing Accuracy: 73.05%
Confusion Matrix:
[[ 9  0]
 [38 94]]
Classification Report:
              precision    recall  f1-score   support

           0       0.19      1.00      0.32         9
           1       1.00      0.71      0.83       132

    accuracy                         

# 4. mRMR Feature Selection

# For 1st Dataset

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
import warnings

warnings.filterwarnings("ignore")

# Load training dataset
df_train = pd.read_csv("ML_DEG_dataset.csv")
columns_of_interest = ['CDH3', 'GNG7', 'GDPD3', 'MMP1', 'ABCA8', 'VSNL1', 'ADH1B', 'AZGP1', 'LGALS2', 'TMEM100', 'DPEP1', 'AFF3', 'BRCA2',
                       'MS4A12', 'CENPA', 'LIFR', 'CEP72', 'ABCG2', 'ABI3BP', 'COL10A1', 'MT1M', 'MAD2L1', 'STMN2', 'NFE2L3', 'SLC4A4', 'KRT23', 'PRR7', 'target']
df_train = df_train[columns_of_interest]

X_train = df_train.iloc[:, :-1]
y_train = df_train.iloc[:, -1]

# Load testing dataset
df_test = pd.read_csv("GSE21815_gene_expression_renamed.csv")
df_test = df_test.iloc[:, 1:]
df_test = df_test[[col for col in df_test.columns if col != 'target'] + ['target']]

X_test = df_test.iloc[:, :-1]
y_test = df_test.iloc[:, -1]

# Find common columns
common_cols = set(X_train.columns).intersection(X_test.columns)
X_train_common = X_train[list(common_cols)]
X_test_common = X_test[list(common_cols)]

# Define base models
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Bagging': BaggingClassifier(),
    'Extra Trees': ExtraTreesClassifier(),
    'XGBoost':XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

# Perform evaluation
seed = 7
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

for name, model in models.items():
    # Cross-validation on training set
    cv_results = cross_val_score(model, X_train_common, y_train, cv=kfold, scoring='accuracy')
    print(f"{name} Training Accuracy: {cv_results.mean() * 100:.2f}%")

    # Train model and test on unseen data
    model.fit(X_train_common, y_train)
    predictions = model.predict(X_test_common)

    # Calculate metrics
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted', zero_division=0)
    recall = recall_score(y_test, predictions, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predictions, average='weighted', zero_division=0)
    mcc = matthews_corrcoef(y_test, predictions)

    # Print results
    print(f"{name} Testing Accuracy: {accuracy * 100:.2f}%")
    print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1-Score: {f1:.2f}, MCC: {mcc:.2f}")

    # Print confusion matrix and classification report
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, predictions))
    print("Classification Report:")
    print(classification_report(y_test, predictions, zero_division=0))
    print("-" * 60)


Logistic Regression Training Accuracy: 93.61%
Logistic Regression Testing Accuracy: 88.65%
Precision: 0.91, Recall: 0.89, F1-Score: 0.90, MCC: 0.22
Confusion Matrix:
[[  3   6]
 [ 10 122]]
Classification Report:
              precision    recall  f1-score   support

           0       0.23      0.33      0.27         9
           1       0.95      0.92      0.94       132

    accuracy                           0.89       141
   macro avg       0.59      0.63      0.61       141
weighted avg       0.91      0.89      0.90       141

------------------------------------------------------------
K-Nearest Neighbors Training Accuracy: 93.31%
K-Nearest Neighbors Testing Accuracy: 90.78%
Precision: 0.96, Recall: 0.91, F1-Score: 0.92, MCC: 0.61
Confusion Matrix:
[[  9   0]
 [ 13 119]]
Classification Report:
              precision    recall  f1-score   support

           0       0.41      1.00      0.58         9
           1       1.00      0.90      0.95       132

    accuracy            

# For 1st Dataset

# 6 . MI Feature Selection

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from xgboost import XGBClassifier
import warnings

warnings.filterwarnings("ignore")

# Load datasets
df_train = pd.read_csv("ML_DEG_dataset.csv")
columns_of_interest = ['ADH1B', 'CDH3', 'GUCA2B', 'PDE9A', 'ABCA8', 'GUCA2A','target']
df_train = df_train[columns_of_interest]
#df_train = df_train.iloc[:, 1:]
df_train = df_train[[col for col in df_train.columns if col != 'target'] + ['target']]
X_train = df_train.iloc[:, 0:-1]
y_train = df_train.iloc[:, -1]

df_test = pd.read_csv("GSE21815_gene_expression_renamed.csv")
df_test = df_test.iloc[:, 1:]
df_test = df_test[[col for col in df_test.columns if col != 'target'] + ['target']]
X_test = df_test.iloc[:, 0:-1]
y_test = df_test.iloc[:, -1]

# Ensure common features in both train and test sets
common_cols = set(X_train.columns).intersection(X_test.columns)
X_train_common = X_train[list(common_cols)]
X_test_common = X_test[list(common_cols)]

# Define models
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Bagging': BaggingClassifier(),
    'Extra Trees': ExtraTreesClassifier(),
    'XGBoost':XGBClassifier(use_label_encoder=False, eval_metric='logloss')

}

# Evaluate models
for name, model in models.items():
    print(f"Evaluating {name}...")

    # Cross-validation accuracy
    kfold = KFold(n_splits=10, shuffle=True, random_state=7)
    cv_results = cross_val_score(model, X_train_common, y_train, cv=kfold, scoring='accuracy')
    print(f"{name} Training Accuracy: {cv_results.mean() * 100:.2f}%")

    # Train and test model
    model.fit(X_train_common, y_train)
    predictions = model.predict(X_test_common)

    # Compute metrics
    test_accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted', zero_division=0)
    recall = recall_score(y_test, predictions, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predictions, average='weighted', zero_division=0)
    mcc = matthews_corrcoef(y_test, predictions)

    print(f"{name} Testing Accuracy: {test_accuracy * 100:.2f}%")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}")
    print(f"MCC: {mcc:.2f}")

    # Print confusion matrix and classification report
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, predictions))
    print("Classification Report:")
    print(classification_report(y_test, predictions))
    print("-" * 60)

Evaluating Logistic Regression...
Logistic Regression Training Accuracy: 94.21%
Logistic Regression Testing Accuracy: 99.29%
Precision: 0.99
Recall: 0.99
F1-Score: 0.99
MCC: 0.94
Confusion Matrix:
[[  8   1]
 [  0 132]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.89      0.94         9
           1       0.99      1.00      1.00       132

    accuracy                           0.99       141
   macro avg       1.00      0.94      0.97       141
weighted avg       0.99      0.99      0.99       141

------------------------------------------------------------
Evaluating K-Nearest Neighbors...
K-Nearest Neighbors Training Accuracy: 93.04%
K-Nearest Neighbors Testing Accuracy: 96.45%
Precision: 0.97
Recall: 0.96
F1-Score: 0.96
MCC: 0.65
Confusion Matrix:
[[  4   5]
 [  0 132]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.44      0.62         9
           1     

# 2nd Dataset

# ElasticNet Feature Selection

# 1st Dataset

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
import warnings

warnings.filterwarnings("ignore")

# Load datasets
df_train = pd.read_csv("ML_DEG_dataset.csv")
columns_of_interest = ['NEBL.1', 'CDH3', 'LGI1', 'BRCA2', 'PTGDS', 'ASPA', 'ABCG2', 'SST', 'TNFRSF11B', 'SLC4A4', 'TAF1A', 'SPINK2', 'PTGDR', 'AFF3', 'DHRS9', 'TG', 'MMP12', 'ADH1B', 'IL23A', 'ACSL6', 'TSPAN7', 'MMP1', 'PDE9A', 'SPINK5', 'CDKN2A','target']
df_train = df_train[columns_of_interest]
#df_train = df_train.iloc[:, 1:]
df_train = df_train[[col for col in df_train.columns if col != 'target'] + ['target']]
X_train = df_train.iloc[:, 0:-1]
y_train = df_train.iloc[:, -1]

df_test = pd.read_csv("GSE21815_gene_expression_renamed.csv")
df_test = df_test.iloc[:, 1:]
df_test = df_test[[col for col in df_test.columns if col != 'target'] + ['target']]
X_test = df_test.iloc[:, 0:-1]
y_test = df_test.iloc[:, -1]

# Ensure common features in both train and test sets
common_cols = set(X_train.columns).intersection(X_test.columns)
X_train_common = X_train[list(common_cols)]
X_test_common = X_test[list(common_cols)]

# Define models
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Bagging': BaggingClassifier(),
    'Extra Trees': ExtraTreesClassifier(),
    'XGBoost':XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

# Evaluate models
for name, model in models.items():
    print(f"Evaluating {name}...")

    # Cross-validation accuracy
    kfold = KFold(n_splits=10, shuffle=True, random_state=7)
    cv_results = cross_val_score(model, X_train_common, y_train, cv=kfold, scoring='accuracy')
    print(f"{name} Training Accuracy: {cv_results.mean() * 100:.2f}%")

    # Train and test model
    model.fit(X_train_common, y_train)
    predictions = model.predict(X_test_common)

    # Compute metrics
    test_accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted', zero_division=0)
    recall = recall_score(y_test, predictions, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predictions, average='weighted', zero_division=0)
    mcc = matthews_corrcoef(y_test, predictions)

    print(f"{name} Testing Accuracy: {test_accuracy * 100:.2f}%")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}")
    print(f"MCC: {mcc:.2f}")

    # Print confusion matrix and classification report
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, predictions))
    print("Classification Report:")
    print(classification_report(y_test, predictions))
    print("-" * 60)

Evaluating Logistic Regression...
Logistic Regression Training Accuracy: 93.03%
Logistic Regression Testing Accuracy: 95.04%
Precision: 0.95
Recall: 0.95
F1-Score: 0.95
MCC: 0.56
Confusion Matrix:
[[  5   4]
 [  3 129]]
Classification Report:
              precision    recall  f1-score   support

           0       0.62      0.56      0.59         9
           1       0.97      0.98      0.97       132

    accuracy                           0.95       141
   macro avg       0.80      0.77      0.78       141
weighted avg       0.95      0.95      0.95       141

------------------------------------------------------------
Evaluating K-Nearest Neighbors...
K-Nearest Neighbors Training Accuracy: 95.35%
K-Nearest Neighbors Testing Accuracy: 98.58%
Precision: 0.99
Recall: 0.99
F1-Score: 0.99
MCC: 0.90
Confusion Matrix:
[[  9   0]
 [  2 130]]
Classification Report:
              precision    recall  f1-score   support

           0       0.82      1.00      0.90         9
           1     